# Beispielfälle für Vasilis

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from l15_msevi import msevi as msv
%matplotlib inline

import datetime as dt
from analysis_tools import grid_and_interpolation as gi
import xarray as xr
import msg_coord_conv2 as mcc

/vols/satellite/home/lenk/anaconda3/envs/tracking/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [2]:
def normalise2range(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def array_to_256(array):
    return (array*255.999).astype("uint8")

def day_natural_composite(vis006_data,vis008_data,nir016_data,factor=1,gamma=1):
    blue = array_to_256(np.clip(vis006_data/factor,0,1)**(1./gamma))
    green = array_to_256(np.clip(vis008_data/factor,0,1)**(1./gamma))
    red = array_to_256(np.clip(nir016_data/factor,0,1)**(1./gamma))
    
    return np.dstack([red,green,blue]).astype("uint8")

def scale_array_min_max(array_data,range_min=0,range_max=1):
    """
    Scales a array into the chosen range.
    
    Inputs:
    -------
    array_data: numpy array of floats or integers, 2d
        array to scale
    range_min: int or float, default = 0
        minimum value of the range to scale array to,
    range_max: int or float, default = 1
        maximum value of the range to scale array to,
        
    Returns:
    --------
    scaled_array: numpy array of floats, 2d
    """
    # get array extrema
    array_min = np.min(array_data)
    array_max = np.max(array_data)

    # derive conversion parameters
    a = (range_max - range_min) / (array_max - array_min)
    b = range_max - a * array_max

    # scale array
    scaled_array = a * array_data + b
    
    return scaled_array

def add_hrv_texture2nc(nc,hrv):
    nc_lab = color.rgb2lab(nc)
    l_min = nc_lab[...,0].min()
    l_max = nc_lab[...,0].max()
    
    l_hrv_scaled = nc_lab[...,0] * hrv
    l_hrv_scaled = scale_array_min_max(l_hrv_scaled,l_min,l_max)

    nc_lab[...,0] = l_hrv_scaled
    
    return color.lab2rgb(nc_lab)

## Koordinatenreferenz

In [3]:
with xr.open_dataset("/vols/satellite/home/lenk/data/SEVIRI/auxdata/msevi_geolocation_rss.nc") as file:
    slon = file.longitude.data
    slat = file.latitude.data

In [104]:
slon.shape

(1392, 3712)

In [4]:
mcc.ll2msg_ij(34,-10,'rss','std')

indices(col=722.9161467971451, row=1289.5168972365664)

In [7]:
mcc.ll2msg_ij(58,-10,'rss','std')

indices(col=218.78317918979133, row=1509.9523055034888)

In [6]:
mcc.ll2msg_ij(34,28,'rss','std')

indices(col=721.8834826837731, row=2394.9527336164983)

In [8]:
mcc.ll2msg_ij(58,28,'rss','std')

indices(col=217.9190689468228, row=2185.1117539169877)

In [16]:
mcc.ll2msg_ij(34,-10,'pzs','std')

indices(col=715.1849931963873, row=1559.370356092671)

In [17]:
mcc.ll2msg_ij(34,28,'pzs','std')

indices(col=733.5970278397394, row=2645.436111970608)

In [18]:
mcc.ll2msg_ij(58,-10,'pzs','std')

indices(col=212.33799292106437, row=1675.284814798284)

In [19]:
mcc.ll2msg_ij(58,28,'pzs','std')

indices(col=227.77920056759604, row=2340.044430905345)

In [96]:
# cmin, rmax = mcc.ll2cl(-10,34,'std','rss','oben')
# cmax, rmin = mcc.ll2cl(28,58,'std','rss','oben')

In [92]:
# cmin = int(np.rint(cmin))
# cmax = int(np.rint(cmax))
# rmin = int(np.rint(rmin))
# rmax = int(np.rint(rmax))

In [14]:
rmin = 218
rmax = 723
cmin = 1290
cmax = 2394

In [11]:
tlist = [dt.datetime(2015,5,11,12,0),
         dt.datetime(2015,6,4,12,0),
         dt.datetime(2015,9,1,12,0),
         dt.datetime(2015,2,1,12,0),
         dt.datetime(2015,1,20,12,0),
         dt.datetime(2015,7,27,12,0)]

In [12]:
channels = ['VIS006','VIS008','IR_016']

In [15]:
for t in tlist:
    try:
        s = msv.MSevi(t,chan_list=channels,region='full',scan_type='rss')
        s.rad2refl()

        nc = day_natural_composite(s.ref['VIS006'][rmin:rmax,cmin:cmax],
                                   s.ref['VIS008'][rmin:rmax,cmin:cmax],
                                   s.ref['IR_016'][rmin:rmax,cmin:cmax],0.9,1.8)

        fig,ax = plt.subplots(1,1,figsize=((cmax - cmin) / 100,
                                            (rmax-rmin) / 100))
        plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
                    hspace = 0, wspace = 0)
        plt.margins(0,0)
        ax.imshow(nc)
        ax.axis('off')
        plt.savefig("/vols/satellite/home/lenk/NC_RGB_{}_cutout.png".format(t.strftime("%Y%m%dT%H%M")),
                    bbox_inches="tight",pad_inches=0,dpi=100)
        plt.close()
    except:
        s = msv.MSevi(t,chan_list=channels,region='full',scan_type='pzs')
        s.rad2refl()

        nc = day_natural_composite(s.ref['VIS006'][rmin:rmax,cmin+300:cmax+300],
                                   s.ref['VIS008'][rmin:rmax,cmin+300:cmax+300],
                                   s.ref['IR_016'][rmin:rmax,cmin+300:cmax+300],0.9,1.8)
        
        fig,ax = plt.subplots(1,1,figsize=((cmax - cmin) / 100,
                                            (rmax-rmin) / 100))
        plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
                    hspace = 0, wspace = 0)
        plt.margins(0,0)
        ax.imshow(nc)
        ax.axis('off')
        plt.savefig("/vols/satellite/home/lenk/NC_RGB_{}_cutout.png".format(t.strftime("%Y%m%dT%H%M")),
                    bbox_inches="tight",pad_inches=0,dpi=100)
        plt.close()

... reading  /tmp/hrit1648015148/H-000-MSG2__-MSG2_RSS____-VIS006___-000006___-201505111200-__
... reading  /tmp/hrit1648015148/H-000-MSG2__-MSG2_RSS____-VIS006___-000007___-201505111200-__
... reading  /tmp/hrit1648015148/H-000-MSG2__-MSG2_RSS____-VIS006___-000008___-201505111200-__
... reading  /tmp/hrit1648015148/H-000-MSG2__-MSG2_RSS____-VIS008___-000006___-201505111200-__
... reading  /tmp/hrit1648015148/H-000-MSG2__-MSG2_RSS____-VIS008___-000007___-201505111200-__
... reading  /tmp/hrit1648015148/H-000-MSG2__-MSG2_RSS____-VIS008___-000008___-201505111200-__
... reading  /tmp/hrit1648015148/H-000-MSG2__-MSG2_RSS____-IR_016___-000006___-201505111200-__
... reading  /tmp/hrit1648015148/H-000-MSG2__-MSG2_RSS____-IR_016___-000007___-201505111200-__
... reading  /tmp/hrit1648015148/H-000-MSG2__-MSG2_RSS____-IR_016___-000008___-201505111200-__

Combine segments

Do calibration
... reading  /tmp/hrit398995677/H-000-MSG2__-MSG2_RSS____-VIS006___-000006___-201506041200-__
... reading  /tmp

... reading  /tmp/hrit4789343516/H-000-MSG2__-MSG2_RSS____-IR_016___-000008___-201507271200-__

Combine segments

Do calibration
